### Imports

In [2]:
import collections
import cPickle as pickle
import itertools
import gc
import json
from matplotlib import pyplot as plt
import numpy as np
import random
import snap
import tools
reload(tools)
# gc.collect()

<module 'tools' from 'tools.pyc'>

### Preprocess

We're going to use SNAP and see how much space it takes. First the node encodings.

- indexToUserId: Maps indexes from 0 to (# users - 1) to the userId
- indexToSubreddit: Maps indexes from 0 to (# subreddits - 1) to the subredditId
- THRESHOLD: All users will have index < THRESHOLD, all subreddits will be stored as THRESHOLD + index. The THRESHOLD will be 5,000,000.

In [3]:
# Load inputGraph, include weights.
inputGraph = collections.defaultdict(lambda: collections.defaultdict(int))
tools.getUserIdToSubreddits("../bigData/finalGeneration/inputGraph", inputGraph, includeCounts=True)
print "Number of users: {}".format(len(inputGraph))

Processing ../bigData/finalGeneration/inputGraph
Processed 1000000
Processed 2000000
Processed 3000000
Processed 4000000
Processed 5000000
Processed 6000000
Processed 7000000
Processed 8000000
Processed 9000000
Processed 10000000
Processed 11000000
Processed 12000000
Processed 13000000
Processed 14000000
Processed 15000000
Processed 16000000
Processed 17000000
Processed 18000000
Processed 19000000
Processed 20000000
Processed 21000000
Processed 22000000
Processed 23000000
Processed 24000000
Processed 25000000
Processed 26000000
Processed 27000000
Processed 28000000
Processed 29000000
Processed 30000000
Processed 31000000
Processed 32000000
Processed 33000000
Processed 34000000
Processed 35000000
Processed 36000000
Processed 37000000
Processed 38000000
Processed 39000000
Processed 40000000
Processed 41000000
Processed 42000000
Processed 43000000
Processed 44000000
Processed 45000000
Processed 46000000
Processed 47000000
Processed 48000000
Processed 49000000
Processed 50000000
Processed 

In [4]:
# Output indexToUserId, intended to be indexed 0 to len(indexToUserIds) - 1
with open("../bigData/pixie/indexToUserId", 'w') as outfile:
    for userId, _ in userIdToOldSubreddits.iteritems():
        outfile.write(userId + "\n")

In [5]:
# Output indexToSubredditId, intended to be indexed 0 to len(indexToSubredditId) - 1
subredditIdSet = set()
for userId, subreddits in userIdToOldSubreddits.iteritems():
    for subreddit in subreddits:
        subredditIdSet.add(subreddit)

with open("../bigData/pixie/indexToSubredditId", 'w') as outfile:
    for subredditId in subredditIdSet:
        outfile.write(subredditId + "\n")

Load the graph into SNAP.

In [4]:
# Load the two mappings.
THRESHOLD = 5000000
indexToUserId, userIdToIndex = tools.loadIndexToUserId("../bigData/pixie/indexToUserId")
indexToSubredditId, subredditIdToIndex = tools.loadIndexToUserId("../bigData/pixie/indexToSubredditId")

inputGraph = snap.TUNGraph.New()
for i in range(len(indexToUserId)):
    inputGraph.AddNode(i)
for i in range(len(indexToSubredditId)):
    inputGraph.AddNode(THRESHOLD + i)
with open("../bigData/finalGeneration/inputGraph", 'r') as f:
    for i, line in enumerate(f, 1):
        userId, subredditId, commentCount = line.split()
        inputGraph.AddEdge(userIdToIndex[userId], THRESHOLD + subredditIdToIndex[subredditId])

        if i % 1000000 == 0:
            print "Processed {}".format(i)

Processed 1000000
Processed 2000000
Processed 3000000
Processed 4000000
Processed 5000000
Processed 6000000
Processed 7000000
Processed 8000000
Processed 9000000
Processed 10000000
Processed 11000000
Processed 12000000


KeyboardInterrupt: 

SNAP too slow, will instead generate two lists of adjacency lists.
- userIndexToSubreddits
- subredditIndexToUsers

This also means we will no longer be using THRESHOLD

In [2]:
# Load the two mappings.
indexToUserId, userIdToIndex = tools.loadIndexToUserId("../bigData/pixie/indexToUserId")
indexToSubredditId, subredditIdToIndex = tools.loadIndexToUserId("../bigData/pixie/indexToSubredditId")

# Throw into lists.
userIndexToSubreddits1 = [[] for i in range(len(indexToUserId))]
subredditIndexToUsers1 = [[] for i in range(len(indexToSubredditId))]
userIndexToSubreddits2 = [[] for i in range(len(indexToUserId))]
subredditIndexToUsers2 = [[] for i in range(len(indexToSubredditId))]
with open("../bigData/finalGeneration/inputGraph", 'r') as f:
    for i, line in enumerate(f, 1):
        userId, subredditId, commentCountStr = line.split()
        userIndex = userIdToIndex[userId]
        subredditIndex = subredditIdToIndex[subredditId]
        commentCount = int(commentCountStr)
        
        userIndexToSubreddits1[userIndex].append(subredditIndex)
        subredditIndexToUsers1[subredditIndex].append(userIndex)
        userIndexToSubreddits2[userIndex].append(commentCount)
        subredditIndexToUsers2[subredditIndex].append(commentCount)
        if i % 1000000 == 0:
            print "Processed {}".format(i)
print "Users: {}, Subreddits: {}".format(len(userIndexToSubreddits1), len(subredditIndexToUsers1))
            
# Output.
with open("../bigData/pixie/userIndexToSubreddits1.pkl", 'w') as outfile:
    pickle.dump(userIndexToSubreddits1, outfile)
with open("../bigData/pixie/subredditIndexToUsers1.pkl", 'w') as outfile:
    pickle.dump(subredditIndexToUsers1, outfile)
with open("../bigData/pixie/userIndexToSubreddits2.pkl", 'w') as outfile:
    pickle.dump(userIndexToSubreddits2, outfile)
with open("../bigData/pixie/subredditIndexToUsers2.pkl", 'w') as outfile:
    pickle.dump(subredditIndexToUsers2, outfile)
print "Done"

Processed 1000000
Processed 2000000
Processed 3000000
Processed 4000000
Processed 5000000
Processed 6000000
Processed 7000000
Processed 8000000
Processed 9000000
Processed 10000000
Processed 11000000
Processed 12000000
Processed 13000000
Processed 14000000
Processed 15000000
Processed 16000000
Processed 17000000
Processed 18000000
Processed 19000000
Processed 20000000
Processed 21000000
Processed 22000000
Processed 23000000
Processed 24000000
Processed 25000000
Processed 26000000
Processed 27000000
Processed 28000000
Processed 29000000
Processed 30000000
Processed 31000000
Processed 32000000
Processed 33000000
Processed 34000000
Processed 35000000
Processed 36000000
Processed 37000000
Processed 38000000
Processed 39000000
Processed 40000000
Processed 41000000
Processed 42000000
Processed 43000000
Processed 44000000
Processed 45000000
Processed 46000000
Processed 47000000
Processed 48000000
Processed 49000000
Processed 50000000
Processed 51000000
Processed 52000000
Users: 4052716, Subre

Pixie works very poorly without biasing the random walk. We will now weigh the random walks by making a list of 100 per user, and 10,000 per subreddit

In [2]:
userIndexToSubreddits1 = None
with open("../bigData/pixie/userIndexToSubreddits1.pkl", 'r') as infile:
    userIndexToSubreddits1 = pickle.load(infile)
subredditIndexToUsers1 = None
with open("../bigData/pixie/subredditIndexToUsers1.pkl", 'r') as infile:
    subredditIndexToUsers1 = pickle.load(infile)
userIndexToSubreddits2 = None
with open("../bigData/pixie/userIndexToSubreddits2.pkl", 'r') as infile:
    userIndexToSubreddits2 = pickle.load(infile)
subredditIndexToUsers2 = None
with open("../bigData/pixie/subredditIndexToUsers2.pkl", 'r') as infile:
    subredditIndexToUsers2 = pickle.load(infile)

Loading subreddit index to user
Loading subreddit index to user


In [4]:
USER_LIST_SIZE = 100
SUBREDDIT_LIST_SIZE = 10000
# indexToUserId, userIdToIndex = tools.loadIndexToUserId("../bigData/pixie/indexToUserId")
# indexToSubredditId, subredditIdToIndex = tools.loadIndexToUserId("../bigData/pixie/indexToSubredditId")

In [6]:
# User -> Subreddits.
userIndexToSubreddits = [[] for i in range(len(userIndexToSubreddits1))]
for userIndex, subredditIndexes in enumerate(userIndexToSubreddits1):
    totalComments = float(sum(userIndexToSubreddits2[userIndex]))
    for subredditIndex, commentCount in zip(subredditIndexes, userIndexToSubreddits2[userIndex]):
        numToAppend = int(np.round(commentCount / totalComments * USER_LIST_SIZE))
        for i in range(numToAppend):
            userIndexToSubreddits[userIndex].append(subredditIndex)
    if (userIndex + 1) % 1000000 == 0:
        print "User: {}".format(userIndex + 1)
sumLengths = reduce(lambda total, nextList: total + len(nextList), userIndexToSubreddits, 0)
print "Users: {}, avgLength: {}".format(len(userIndexToSubreddits), sumLengths / float(len(userIndexToSubreddits)))

# Dump.
with open("../bigData/pixie/userIndexToSubreddits.pkl", 'w') as outfile:
    pickle.dump(userIndexToSubreddits, outfile)
print "Done"
userIndexToSubreddits = None

User: 999999
User: 1999999
User: 2999999
User: 3999999
Users: 4052716, avgLength: 99.6149278657
Done


In [5]:
# Subreddits -> User.
subredditIndexToUsers = [[] for i in range(len(subredditIndexToUsers1))]
for subredditIndex, userIndexes in enumerate(subredditIndexToUsers1):
    totalComments = float(sum(subredditIndexToUsers2[subredditIndex]))
    for userIndex, commentCount in zip(userIndexes, subredditIndexToUsers2[subredditIndex]):
        numToAppend = int(np.round(commentCount / totalComments * SUBREDDIT_LIST_SIZE))
        for i in range(numToAppend):
            subredditIndexToUsers[subredditIndex].append(userIndex)
    if (subredditIndex + 1) % 10000 == 0:
        print "Subreddit: {}".format(subredditIndex + 1)
sumLengths = reduce(lambda total, nextList: total + len(nextList), subredditIndexToUsers, 0)
print "Subreddits: {}, avgLength: {}".format(len(subredditIndexToUsers), sumLengths / float(len(subredditIndexToUsers)))

# Dump.
with open("../bigData/pixie/subredditIndexToUsers.pkl", 'w') as outfile:
    pickle.dump(subredditIndexToUsers, outfile)
print "Done"
subredditIndexToUsers = None

Subreddit: 10000
Subreddit: 20000
Subreddit: 30000
Subreddit: 40000
Subreddit: 50000
Subreddits: 54204, avgLength: 9947.33427422
Done


### Running Pixie

Really just for testing it, since we'd like to keep the whole graph in memory for a while.

In [3]:
import pixie
import os
import time

In [4]:
# Load everything.
print "Loading user id indexes"
indexToUserId, userIdToIndex = tools.loadIndexToUserId("../bigData/pixie/indexToUserId")
print "Loading subreddit id indexes"
indexToSubredditId, subredditIdToIndex = tools.loadIndexToUserId("../bigData/pixie/indexToSubredditId")
print "Loading user id to old subreddits"
userIdToOldSubreddits = tools.getUserIdToSubredditsByType("../bigData/finalGeneration/expUsers", "oldSubreddits")
print "Loading user id to new subreddits"
userIdToNewSubreddits = tools.getUserIdToSubredditsByType("../bigData/finalGeneration/expUsers", "newSubreddits")
print "Loading subreddit id to name"
subredditIdToName = tools.read_subreddit_names("../bigData/subredditIdToName")

Loading user id indexes
Loading subreddit id indexes
Loading user id to old subreddits
Loading user id to new subreddits
Loading subreddit id to name
Loading user index to subreddits


In [ ]:
print "Loading user index to subreddits"
userIndexToSubreddits = None
with open("../bigData/pixie/userIndexToSubreddits.pkl", 'r') as infile:
    userIndexToSubreddits = pickle.load(infile)
print "Loading subreddit index to user"
subredditIndexToUsers = None
with open("../bigData/pixie/subredditIndexToUsers.pkl", 'r') as infile:
    subredditIndexToUsers = pickle.load(infile)

In [5]:
# Reload module and set seeds.
reload(pixie)
random.seed(7224)
np.random.seed(7224)

# Parameters.
N = 100000
alpha = 0.5

# Get recs.
precision10 = 0
mrrMetric = 0
mapMetric = 0
numUsers = 0
for i, (userId, oldSubreddits) in enumerate(userIdToOldSubreddits.iteritems()):    
    print "Processing {} {}".format(i, userId)
    recs = pixie.getRecs(oldSubreddits,
                         subredditIdToIndex,
                         indexToSubredditId,
                         userIndexToSubreddits,
                         subredditIndexToUsers,
                         N, None, alpha)
    
    # Precision at 10.
    goodRec10Count = 0
    for _, rec in recs[:10]:
        if rec in userIdToNewSubreddits[userId]:
            goodRec10Count += 1
    precision10 += goodRec10Count / 10.
    
    # Mean Reciprocal Rank.
    for rank, (_, rec) in enumerate(recs, 1):
        if rec in userIdToNewSubreddits[userId]:
            mrrMetric += 1. / rank
            break
    
    # Mean Average Precision.
    correctSoFar = 0
    numActual = float(len(userIdToNewSubreddits[userId]))
    for rank, (_, rec) in enumerate(recs, 1):
        if rec in userIdToNewSubreddits[userId]:
            correctSoFar += 1
            mapMetric += correctSoFar / numActual / rank

totalUsers = float(len(userIdToOldSubreddits))
print "N: {}, alpha: {}, precision@10: {}, mrr: {}, map: {}".format(
    N,
    alpha,
    precision10 / totalUsers,
    mrrMetric / totalUsers,
    mapMetric / totalUsers)

Processing 0 elizle
Processing 1 spikespaz
Processing 2 Woodpecker16669
Processing 3 Dumpstertrash1
Processing 4 Avalollk
Processing 5 slickslink
Processing 6 Zhangathan_Jon
Processing 7 mooniesoloonie
Processing 8 TorreTiger25
Processing 9 Deenyc43
Processing 10 thomasmagnum
Processing 11 LangourDaydreams
Processing 12 ChaosOnion
Processing 13 Deridex3101
Processing 14 absolince
Processing 15 farkhipov
Processing 16 UnvaccinatedAutist
Processing 17 pandillasexo
Processing 18 show_me_the
Processing 19 elbowe21
Processing 20 BigAbbott
Processing 21 AssasinButt
Processing 22 Danosaur42089
Processing 23 amskeez
Processing 24 Scissor_Runner12
Processing 25 luck_panda
Processing 26 coheedcollapse
Processing 27 thomasd4nkengine
Processing 28 BelievingEal21


pixie.py:86: RuntimeWarning: divide by zero encountered in log
  scalingFactors.append(oldSubredditDegree * (maxSubredditDegree - np.log(oldSubredditDegree)))
pixie.py:86: RuntimeWarning: invalid value encountered in double_scalars
  scalingFactors.append(oldSubredditDegree * (maxSubredditDegree - np.log(oldSubredditDegree)))


Processing 29 bongtokent
Processing 30 wanderershe-ra
Processing 31 Brunsy89
Processing 32 BrownMan97
Processing 33 art_hoe1
Processing 34 FF_Firefighter
Processing 35 harpuajim25
Processing 36 Pro_phet
Processing 37 iamliterallysatan
Processing 38 GTL5427
Processing 39 FocusedADD
Processing 40 habibiiiiiii
Processing 41 Benjii117
Processing 42 ZombieJesus1987
Processing 43 PussyNoodle
Processing 44 msammy07
Processing 45 Vag_Assasin
Processing 46 Sockoram
Processing 47 nerveless
Processing 48 hurricane1197
Processing 49 jodiparks
Processing 50 Miiiiinx
Processing 51 KingOfNiFe
Processing 52 aN1mosity_
Processing 53 FoxClass
Processing 54 The_Fun_Sized
Processing 55 Pardnerr
Processing 56 gueriLLaPunK
Processing 57 DrainTheMuck
Processing 58 archivalerie
Processing 59 LazyPrinciple
Processing 60 Slayer_Blake
Processing 61 TEKNOC
Processing 62 MarvelRay
Processing 63 HyruleHeroLink
Processing 64 UnrequitedLovecraft
Processing 65 TheCodGamer
Processing 66 Sw0rDz
Processing 67 lostinthera

In [4]:
numEdges = 0
for users in subredditIndexToUsers:
    numEdges += len(users)
print numEdges

18716761


In [86]:
# Test out timing here.
start = time.time()
currentSubreddit = 0
rands = np.random.random((1000000,))
for i in range(100000):
    rand1 = int(len(subredditIndexToUsers[currentSubreddit]) * rands[i])
    currentUser = subredditIndexToUsers[currentSubreddit][rand1]
    rand2 = int(len(userIndexToSubreddits[currentUser]) * rands[10000 + i])
    currentSubreddit = userIndexToSubreddits[currentUser][rand2]
    # currentUser = np.random.choice(subredditIndexToUsers[currentSubreddit])
    # currentSubreddit = np.random.choice(userIndexToSubreddits[currentUser])
print time.time() - start



0.300087213516


In [10]:
print inputGraph[indexToUserId[1]]

defaultdict(<type 'int'>, {'t5_2sfmf': 4, 't5_2th52': 7, 't5_2qzb6': 2, 't5_2qh33': 6, 't5_2rbod': 16, 't5_2qmg3': 17, 't5_2s28b': 6, 't5_2qjfk': 3, 't5_2qjyy': 37, 't5_2qh03': 3, 't5_37mcm': 1, 't5_3j2oi': 4, 't5_2rebv': 22, 't5_2qh1u': 2, 't5_2qh3u': 26, 't5_2x5s1': 2, 't5_2r8rv': 4, 't5_2qgzy': 19, 't5_2tk95': 1, 't5_3itci': 15, 't5_2qh1e': 2, 't5_2s5oq': 2, 't5_2si5v': 3, 't5_2s4yk': 4, 't5_2qh3l': 8, 't5_2qh1i': 15, 't5_3kfwf': 2, 't5_2s91l': 2, 't5_2qt55': 7, 't5_39usd': 20, 't5_2qqjc': 6, 't5_2qi6d': 3, 't5_2cneq': 44, 't5_2qh6e': 1, 't5_32xq7': 1, 't5_2qmah': 2, 't5_2qnts': 4, 't5_2qh3s': 6, 't5_3f5iq': 6, 't5_2qh0u': 8, 't5_34em3': 5, 't5_2qh13': 5, 't5_2qig7': 35, 't5_mouw': 4, 't5_2qxqc': 2, 't5_2rjpr': 25, 't5_21of': 1, 't5_2t160': 7, 't5_3fuwj': 9, 't5_2qkg3': 1, 't5_35g9w': 1, 't5_2qhx4': 1, 't5_2sokd': 1, 't5_2x66f': 9, 't5_3a1d4': 1})


### Qualitative Results

In [6]:
actualNewSubreddits = tools.getUserIdToSubredditsByType("../bigData/finalGeneration/actualNewSubreddits", "newSubreddits")

Processed 1000000
Processed 2000000


In [9]:
total = 0
for _, newSubreddits in actualNewSubreddits.iteritems():
    total += len(newSubreddits)
print total

6797506


In [3]:
print "Loading subreddit id to name"
subredditIdToName = tools.read_subreddit_names("../bigData/subredditIdToName")
subredditNameToId = {node_id: node for node, node_id in subredditIdToName.iteritems()}

Loading subreddit id to name


In [6]:
subredditNames = ['ShinyPokemon', 'PokemonPlaza', 'pokemon', 'PokemonCreate', 'CasualPokemonTrades', 'battleagency', 'pokemontrades', 'PokeMoonSun', 'SVExchange', 'friendsafari', 'FestivalPlaza', 'pokemonshowdown', 'pokemonrng', 'ANormalDayintheSims', 'stunfisk', 'childtheories', 'Pokemonexchange', 'OrbitingSphere', 'Pokemongiveaway', 'SourBuizel', 'richarddawkins', 'watermelon', 'historyboners', 'PokemonSunAndMoon', 'LemonySnicket', 'ConfessionBear', 'detectivepikachu', 'pokemonribbons', 'neildegrassetyson', 'WonderTrade', 'petpeeve', 'GrindsMyGears', 'PokemonRejuvenation', 'powersaves', 'PokemonFusion', 'PokeTube', 'bravewilderness', 'kidfree', 'powersaves3ds', 'GodlessWomen', 'pokespe', 'PokemonPokedexHelp', 'letterswap', 'rosesarered', 'pokemonexchangeref', 'Pokemon_Giveaways', 'Cookies', 'Pokemonguide', 'planetarysociety', 'sens', 'pokemonteams', 'PokemonConquest', 'pokemonXY', 'PokemonORAS', 'SoloPokes', 'pokemondraftleague', 'Winniethepooh', 'Muffins', 'twom', 'Ambipom', 'friendsafaricodes']
subredditIds = [subredditNameToId[subredditName] for subredditName in subredditNames]
for i, (userId, oldSubreddits) in enumerate(inputGraph.iteritems(), 1):
    if userId not in actualNewSubreddits:
        continue
    newSubreddits = actualNewSubreddits[userId]
    if len(newSubreddits) > 100:
        continue
    
    numTotal = len(subredditIds)
    numInOld = 0
    numInNew = 0
    for subredditId in subredditIds:
        if subredditId in oldSubreddits:
            numInOld += 1
        if subredditId in newSubreddits:
            numInNew += 1
    if numInOld >= 4 and numInNew >= 3:
        print userId
    if i % 1000000 == 0:
        print "User {}".format(i)
    
    

User 1000000
Jloi2001
User 2000000
User 3000000
ArchaicArmaldo
User 4000000


In [7]:
print [subredditIdToName[thing] for thing in inputGraph["Jloi2001"]]
print [subredditIdToName[thing] for thing in actualNewSubreddits["Jloi2001"]]

['CasualPokemonTrades', 'PokemonPlaza', 'Pokemongiveaway', 'pokemontrades', 'pokemon']
['SVExchange', 'relaxedpokemontrades', 'Pokemonexchange', 'pokemonrng', 'PokeMoonSun']


In [8]:
with open('../bigData/finalGeneration/user1', 'w') as outfile:
    for userId in ["Jloi2001"]:
        userJson = {"userId": userId, 
                    "newSubreddits": actualNewSubreddits[userId],
                    "oldSubreddits": inputGraph[userId]}
        outfile.write(json.dumps(userJson) + "\n")

In [12]:
a = ['t5_3csff', 't5_2qh1i', 't5_3ggz0', 't5_2se9w', 't5_2z47n', 't5_2qh03', 't5_2yt52', 't5_2ukac', 't5_4bnoh', 't5_2sn6d']
b = [subredditIdToName[thing] for thing in a]
line = ""
for thing in b:
    line += " {} ".format(thing)
print line



 PokeMoonSun  AskReddit  PokemonCreate  ShinyPokemon  SVExchange  gaming  friendsafari  Pokemonexchange  battleagency  stunfisk 
